In [ ]:
!pip install backtrader yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

plt.switch_backend('Agg')


# Define the Moving Average Crossover Strategy
class MovingAverageCrossover(bt.Strategy):
    # Parameters for the short and long moving averages
    params = (("short_period", 20), ("long_period", 200),)

    def __init__(self):
        # Define short-term moving average (e.g., 20-day SMA)
        self.short_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.short_period
        )
        # Define long-term moving average (e.g., 200-day SMA)
        self.long_ma = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.long_period
        )
        # Define a crossover indicator to detect when the short MA crosses the long MA
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def next(self):
      print(f"Short MA: {self.short_ma[0]}, Long MA: {self.long_ma[0]}")
      if np.isfinite(self.short_ma[0]) and np.isfinite(self.long_ma[0]):
          if self.crossover > 0:  # Buy signal
              self.buy()
          elif self.crossover < 0:  # Sell signal
              self.sell()

# Function to fetch data from Yahoo Finance
def fetch_data(ticker, start, end):
    # Download historical data for the specified ticker and date range
    data = yf.download(ticker, start=start, end=end)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # Replace any infinite values with NaN
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Drop any rows with NaN values to ensure data integrity
    data.dropna(inplace=True)

    # Rename columns to match the format expected by Backtrader
    data = data.rename(columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    })

    # Arrange columns in the specific order required by Backtrader
    data = data[['open', 'high', 'low', 'close', 'volume']]

    # Convert column names to strings to avoid any potential issues with column handling
    data.columns = [str(col) for col in data.columns]

    # Drop rows with NaN values again (after renaming and reordering columns)
    data = data.dropna()

    return data

# Fetch historical data for a specific ticker and date range
ticker = "AAPL"  # Set your desired ticker symbol
# Extend data range to cover moving average calculations
start_date = "2018-01-01"  # Provides buffer period for the 200-day moving average
end_date = "2023-01-01"
data = fetch_data(ticker, start_date, end_date)

# Check for NaN or Inf values in the data
print("NaN or Inf Summary:\n", data.isna().sum())
print("\nData Sample:\n", data.head())

# Display the structure and data types to ensure compatibility with Backtrader
print("DataFrame Structure:\n", data.head())
print("\nColumn Data Types:\n", data.dtypes)

# Prepare the data feed for Backtrader by using PandasData wrapper
data_feed = bt.feeds.PandasData(dataname=data)

# Initialize the Cerebro engine for backtesting
cerebro = bt.Cerebro()

# Add the MovingAverageCrossover strategy to Cerebro
cerebro.addstrategy(MovingAverageCrossover)

# Add the data feed to Cerebro
cerebro.adddata(data_feed)

# Set initial capital for the backtest
cerebro.broker.setcash(10000)

# Define the position size (number of shares per trade)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Print the starting portfolio value before running the backtest
print("Starting Portfolio Value:", cerebro.broker.getvalue())

# Run the backtest with the specified data and strategy
cerebro.run(exactbars=-1)

# Print the ending portfolio value after the backtest
print("Ending Portfolio Value:", cerebro.broker.getvalue())

# Plot the backtest results including price data and buy/sell signals
fig = cerebro.plot(iplot=False, figsize=(12, 8), dpi=100)[0][0]
fig.savefig("backtest_result.png")

[*********************100%***********************]  1 of 1 completed


NaN or Inf Summary:
 open      0
high      0
low       0
close     0
volume    0
dtype: int64

Data Sample:
                                 open       high        low      close  \
Date                                                                    
2018-01-02 00:00:00+00:00  42.540001  43.075001  42.314999  43.064999   
2018-01-03 00:00:00+00:00  43.132500  43.637501  42.990002  43.057499   
2018-01-04 00:00:00+00:00  43.134998  43.367500  43.020000  43.257500   
2018-01-05 00:00:00+00:00  43.360001  43.842499  43.262501  43.750000   
2018-01-08 00:00:00+00:00  43.587502  43.902500  43.482498  43.587502   

                              volume  
Date                                  
2018-01-02 00:00:00+00:00  102223600  
2018-01-03 00:00:00+00:00  118071600  
2018-01-04 00:00:00+00:00   89738400  
2018-01-05 00:00:00+00:00   94640000  
2018-01-08 00:00:00+00:00   82271200  
DataFrame Structure:
                                 open       high        low      close  \
Date       